<a href="https://colab.research.google.com/github/SteveMama/avatar/blob/master/Avatarify_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pre-requisites needed


1.   Go to: https://dashboard.ngrok.com/login
2.   Signup and copy the authtoken to the clipboard
3.   if using windows, install conda and the github link
https://github.com/SteveMama/avatar.git
4.   Open anaconda prompt and move into the avatar-master or avatar directory



### Now lets dive into colab 

### Avatarify
Follow the steps below to clone Avatarify and install the dependencies.

In [2]:
!cd /content
!rm -rf *

In [3]:
!git clone https://github.com/SteveMama/avatar.git

Cloning into 'avatar'...
remote: Enumerating objects: 1275, done.
remote: Counting objects: 100% (1275/1275), done.
remote: Compressing objects: 100% (414/414), done.
remote: Total 1275 (delta 853), reused 1268 (delta 850), pack-reused 0
Receiving objects: 100% (1275/1275), 5.66 MiB | 22.38 MiB/s, done.
Resolving deltas: 100% (853/853), done.


In [5]:
cd avatar

/content/avatar


In [6]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy pyyaml==5.1

Cloning into 'fomm'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (211/211), 58.16 MiB | 27.64 MiB/s, done.
Resolving deltas: 100% (108/108), done.
     |████████████████████████████████| 276kB 4.3MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=cd367a4a7733d37a420c12b0a3f4a5448e836f0fffbb4948b931bf8610e10146
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
!scripts/download_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228M  100  228M    0     0  69.3M      0  0:00:03  0:00:03 --:--:-- 69.3M
Expected checksum: 8a45a24037871c045fbb8a6a8aa95ebc
Found checksum:    8a45a24037871c045fbb8a6a8aa95ebc  vox-adv-cpk.pth.tar


### ngrok
Follow the steps below to setup ngrok. You will also need to sign up on the ngrok site and get your authtoken (free).


In [20]:
# Download ngrok

!/content/avatar/scripts/get_ngrok.sh

# Run
Start here if the runtime was restarted after installation.

In [22]:
cd /content/avatar

/content/avatar


In [23]:
#!git pull origin

In [24]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [25]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Start the worker


In [26]:
# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

This command should print lines if the worker is successfully started

In [27]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root         340 37.5  2.1 2065168 282456 ?      S    08:40   0:02 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         355  0.0  1.1 2073364 152444 ?      Sl   08:40   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         358  0.0  1.1 2065168 149876 ?      S    08:40   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         359  0.0  1.1 2073364 152440 ?      Sl   08:40   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-po

# Open ngrok tunnel

#### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [28]:
# Paste your authtoken here in quotes
authtoken = "YOUR AUTH TOKEN HERE"

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [29]:
# Set your region here in quotes
region = "us"

In [30]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [31]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [32]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

tcp://2.tcp.ngrok.io:15938 -> 5557 [input]
tcp://2.tcp.ngrok.io:11705 -> 5558 [output]
Tunnel opened


In [33]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Start the client
When you run the cell below it will print a command. Run this command on your computer:

1. Open a terminal (in Windows open `Anaconda Prompt`);
2. Change working directory to the `avatarify` or `avatar-master` directory:</br>
* Windows (change `C:\path\to\avatarify` or `C:\path\to\avatar-master` to your path)</br>
`cd C:\path\to\avatar-master`</br></br>

3. Copy-paste to the terminal the command below and run;
4. It can take some time to connect (usually up to 10 seconds). If the preview window doesn't appear in a minute or two, look for the errors above in this notebook and report in the [issues](https://github.com/alievk/avatarify/issues) or [Slack](https://join.slack.com/t/avatarify/shared_invite/zt-dyoqy8tc-~4U2ObQ6WoxuwSaWKKVOgg).

In [34]:
print('Copy-paste to the terminal the command below and run (press Enter)\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

Copy-paste to the terminal the command below and run (press Enter)

Mac:
./run_mac.sh --is-client --in-addr tcp://2.tcp.ngrok.io:15938 --out-addr tcp://2.tcp.ngrok.io:11705

Windows:
run_windows.bat --is-client --in-addr tcp://2.tcp.ngrok.io:15938 --out-addr tcp://2.tcp.ngrok.io:11705

Linux:
./run.sh --is-client --in-addr tcp://2.tcp.ngrok.io:15938 --out-addr tcp://2.tcp.ngrok.io:11705


#After executing the commmand line in anaconda Prompt at the Avatarify directory, you can change between avatars using keys (1-9)

# Logs

If something doesn't work as expected, please run the cells below and include the logs in your report.

In [35]:
#@title
!cat ./var/log/cam_fomm.log | head -100

[1594543253.563415] Loading Predictor


In [36]:
#@title
!cat ./var/log/recv_worker.log | tail -100

[1594543255.413245] Receiving on port 5557


In [37]:
#@title
!cat ./var/log/predictor_worker.log | tail -100

In [38]:
#@title
!cat ./var/log/send_worker.log | tail -100

[1594543255.419916] Sending on port 5558
